# Biostatistical analysis

We want to dig further the analysis on the cancer risk facotrs and try to quantify their effect, along with other clinical and biological features. To that end, we will compare the population with cancer to the control population. 
<br> We will want to build a table as follow : 


|   | Population without cancer   | Population with cancer  | p-value  |
|---|---|---|---|
| age, mean (IQR)  |  xxx | xxx  | xxx  |
| men, % | xxx  | xxx  | xxx  |
| tabac patients, %  | xxx  | xxx  | xxx  |
| diabetes patients, % | xxx  | xxx  | xxx  |
| alcool patients, % | xxx  | xxx  | xxx  |
| sub_psy patients, % | xxx  | xxx  | xxx  |
| tum_herit patients, % | xxx  | xxx  | xxx  |
| bmi, mean (IQR)  |  xxx | xxx  | xxx  |
| crp, mean (IQR)  |  xxx | xxx  | xxx  |
| urea, mean (IQR)  |  xxx | xxx  | xxx  |
| Hemoglobine, mean (IQR)  |  xxx | xxx  | xxx  |


The p-value column compares, for each caracteristic, the population 1 and the population 2. 
<br> We will admit that a p-value under 0.05 is significant.

The following libraries are required to go through this notebook:

In [2]:
import pandas as pd
import numpy as np

# For descriptive statistics
from scipy.stats import ttest_ind, chi2_contingency

# For Logistic Models implementation
from statsmodels.formula.api import logit, glm

# Table of content

1. [Preprocessing](#preprocessing)  
    1.1 [Age](#age)  
    1.2 [Gender](#gender)  
    1.3 [Comorbidities](#comorb)  
    1.4 [Biological values](#bio)  
    1.5 [Medication data](#med)
2. [Descriptive statistics](#stat)  

4. [Takeaways](#takeaways)


<a id="preprocessing"></a>
# 1. Preprocessing

The main objective of this step is to prepare a dataset for each cohort with the characteristics we need to compare the 2 cohorts.

Hence, we will create a DataFrame with one row by patient, having the following columns:

- age at the time of the visit (*int*)
- male (*bool*)
- comorbidities : cancer antecedent and diabetes (*bool*)
- biological profile at the time of admission (*float*). The following items were measured for each patient : bmi, urea, hemoglobin, crp.
- medication

Then, we will split this dataframe in two :  we will create a ```pd.dataframe``` *df_control* and a ```pd.dataframe``` *df_drugA*.

**NB: For the sake of simplicity, we assume that all the available tables have been pre-processed (deduplication, NLP enhancing , data cleaning, ...) and can be used as is. Moreover, we assume that each patient has one and only visit associated.**

Open the following files using the `pandas.read_pickle()` function : 
  - *data/df_person.pkl* as `df_person`
  - *data/df_visit.pkl* as `df_visit`
  - *data/df_condition.pkl* as `df_cond`
  - *data/df_med.pkl* as `df_med`
  - *data/df_bio.pkl* as `df_bio.pkl`

In [3]:
# Patients
df_person = pd.read_pickle('data/df_person.pkl')
# Visits
df_visit = pd.read_pickle('data/df_visit.pkl')
# Diagnosis (condition)
df_condition = pd.read_pickle('data/df_condition.pkl')
# Medication
df_med = pd.read_pickle('data/df_med.pkl')
# Biology
df_bio = pd.read_pickle('data/df_bio.pkl')

Create the base output dataset (format : `pandas.DataFrame` ; name : `data`)  from the `df_person` table, gathering patient id, gender, birth and death dates.

In [4]:
data = df_person[["person_id", "birth_datetime", "gender_source_value" ,"death_datetime"]]

Check the size of the dataframe using `dataframe.shape()`

In [5]:
data.shape

(18900, 4)

<a id="age"></a>
## 1.1 Age

Compute the patient age at the time of their hospitalization.

TIPS : 
- Inner join the `data` dataset and the `df_visit` dataset on `person_id` using the `pandas.merge()` function (see [doc](https://pandas.pydata.org/docs/reference/api/pandas.merge.html))
- Create an `age` column compute the age for each patient.

In [6]:
data = pd.merge(data, df_visit[["person_id", "visit_occurrence_id" ,"visit_start_datetime"]], on="person_id", how="inner")

In [7]:
data['age'] = round((data['visit_start_datetime'] - data['birth_datetime']).dt.days/ 365,2)

<a id="gender"></a>
## 1.2 Gender

Create a "male" boolean column to identify if the patient is a male (1) or a female (0).

In [8]:
data["male"] = data.gender_source_value == "m"

<a id="comorb"></a>
## 1.3 Comorbidities : history of cancer and diabetes

Meetings with experts lead us to identify the ICD-10 codes corresponding to history of cancer and diabetes :
- cancer : ["Z851", "Z852", "Z853"]
- diabetes : ["E10", "E11", "E12"]

Tag each code in the `df_cond` Dataframe corresponding to diabetes or cancer antecedent.

TIP: you can build a `code_to_comorb` dictionary mapping each ICD-10 code to its corresponding comorbidity, and use the pandas `df.col_name.map(dict)` built-function to map the values of the `condition_source_value` column.

In [9]:
comorb_to_code = {
    "diabetes": ["E10", "E11", "E12"],
    "history_cancer": ["Z851", "Z852", "Z853"]
}

In [10]:
code_to_comorb = {}
for key,codes_list in comorb_to_code.items():
    for code in codes_list:
        code_to_comorb[code]=key

In [11]:
df_condition["comorbidity"] = df_condition.condition_source_value.map(code_to_comorb)

Create a `df_comorbidities` DataFrame by selecting the condition codes corresponding to the desired comorbidities.

TIP : 
You can use the `df.col_name.notna()` built-in function : 
```python 
df[df.col_name.notna()]
```

In [12]:
df_comorbidities = df_condition[df_condition.comorbidity.notna()]

How many patients have cancer antecedent ? How many have had diabetes ?

TIP : Use the `df.col_name.value_counts()` function

In [13]:
df_comorbidities.comorbidity.value_counts()

diabetes          1866
history_cancer     296
Name: comorbidity, dtype: int64

In [14]:
df_comorbidities.head()

,visit_occurrence_id,person_id,condition_occurrence_id,condition_source_value,comorbidity
0,88010689,83375511,86985408,Z851,history_cancer
0,83975057,86414542,84906736,Z853,history_cancer
0,86363739,82584695,81617843,E11,diabetes
1,84473271,83776040,82847145,E11,diabetes
2,88673465,86301131,86041258,E11,diabetes


The `df_comorbidities` DataFrame is composed of one row per condition source value (either cancer or diabete).
<br> A patient can have multiple rows depending on how many conditions they have.

Now, we will create a dataframe with one row per patient and per visit : 
<br>NB : each patient only has one and only visit

| visit_occurrence_id | person_id  | history_cancer | diabetes |
| :------------------ | :----------|:------------| :---------- |
| aaa | xxx | True |  True |
| bbb | yyy | False |  True |
| ccc | zzz | False |  False |

Pivot the `df_comorbidities` DataFrame to have the presence of both antecedents for each patient. Don't forget to replace the NaN values by False and to reset the index !

TIP 
You can use the [pivot](https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.pivot.html) built-in function :
```python
df.pivot(index=["index_col"], columns=["col"], values=["value"])
```
You might need to create "artificially" the *value* column as a True boolean, and drop the *condition_occurrence_id* column.

In [15]:
df_comorbidities["value"] = True

c:\Users\gazze\anaconda3\envs\eds-tuto\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [16]:
df_comorbidities = df_comorbidities.drop(columns=["condition_occurrence_id"]).pivot(index=["person_id", "visit_occurrence_id"], columns=["comorbidity"], values="value").reset_index().fillna(False)

In [17]:
df_comorbidities.head()

comorbidity,person_id,visit_occurrence_id,diabetes,history_cancer
0,80006723,82844300,True,False
1,80006943,88194497,True,False
2,80008962,87210142,True,False
3,80014184,82732761,True,False
4,80015386,82248945,True,False


In [18]:
df_comorbidities.shape

(2112, 4)

In [19]:
df_person.shape

(18900, 5)

Merge the `data` DataFrame with the obtained `df_comorbidities` to summarize comorbidities information for each patient.

TIP:  
- Think twice about the type of merging you want to perform
- Do not forget to fill NA values for the comorbidity columns with False !

In [20]:
data = pd.merge(data, df_comorbidities[["person_id", "history_cancer", "diabetes"]], on="person_id", how="left").fillna({"history_cancer":False, "diabetes":False})

In [21]:
data.head()

,person_id,birth_datetime,gender_source_value,death_datetime,visit_occurrence_id,visit_start_datetime,age,male,history_cancer,diabetes
0,89340407,2024-03-06,f,NaT,84672033,2024-06-27,0.309589,False,False,False
1,89954933,2019-04-26,f,2020-12-22,82105428,2020-12-19,1.652055,False,False,False
2,86079819,2016-10-13,f,NaT,86564132,2021-03-14,4.419178,False,False,False
3,80919213,2021-04-23,f,NaT,84702833,2024-02-09,2.800000,False,False,False
4,82173885,2021-09-10,f,NaT,85134636,2021-11-20,0.194521,False,False,False


In [22]:
data.diabetes.sum()

1866

<a id="bio"></a>
## 1.4 Biological profile

Check the different biological values available.

In [23]:
df_bio.concept_source_value.value_counts()

bmi     18900
urea    18900
crp     18900
hb      18900
Name: concept_source_value, dtype: int64

Create a `df_bio_processed` DataFrame with all biological values for each visit : 

| visit_occurrence_id | bio_A  | bio_B | bio_C |
| :------------------ | :----------|:------------| :---------- |
| aaa | 189 | 13 |  0.87 |
| bbb | 160 | 11 |  0.59 |
| ccc | 209 | 11 |  0.69 |


TIP : Pivot the `df_bio` DataFrame and retrieve the biological values measured at the admission of each hospitalization.

In [24]:
bio_processed = df_bio[["visit_occurrence_id", "concept_source_value", "transformed_value"]].pivot(index=["visit_occurrence_id"], columns="concept_source_value", values="transformed_value").reset_index()

In [25]:
bio_processed.head()

concept_source_value,visit_occurrence_id,bmi,crp,hb,urea
0,80000613,20.05,4.27,13.36,1.98
1,80001324,27.83,4.53,14.88,2.27
2,80001693,28.70,4.28,13.75,1.96
3,80001776,24.07,4.73,13.91,1.19
4,80002034,26.55,4.92,15.52,1.99


Merge `data` and `bio_processed` on `visit_occurrence_id`

TIP: Think twice about the type of merging you want to perform

In [26]:
data = pd.merge(data, bio_processed, on="visit_occurrence_id", how="left")

<a id="med"></a>
## 1.5 Medication taken

Create a "drugA" boolean column in `data` indicating whether patients have taken the drugA or not. 

TIP :
You can pivot the `df_med` DataFrame as done for the comorbidities.

In [27]:
df_med["value"]=True

In [28]:
med_processed = df_med[["visit_occurrence_id", "drug_source_value", "value"]].pivot(index=["visit_occurrence_id"], columns="drug_source_value", values="value").reset_index().fillna(False)

In [29]:
data = pd.merge(data, med_processed, on="visit_occurrence_id", how="left").fillna({"drugA":False})

### Final step

Select the columns you want to keep for the statistical analyses and check that you don't have missing values.

TIP : you can use the following built-in function :
```python
df.isna().sum()
```
Beware of the axis along which you sum !

In [30]:
data = data[["person_id", "age", "male", "history_cancer", "diabetes", "bmi", "crp", "urea", "hb", "drugA"]]

In [31]:
data.isna().sum(axis=0)
data.head()

,person_id,age,male,history_cancer,diabetes,bmi,crp,urea,hb,drugA
0,89340407,0.309589,False,False,False,19.68,4.81,1.25,14.86,False
1,89954933,1.652055,False,False,False,18.75,4.62,0.72,12.55,False
2,86079819,4.419178,False,False,False,20.21,6.07,0.84,11.05,False
3,80919213,2.800000,False,False,False,16.95,4.35,1.25,12.45,False
4,82173885,0.194521,False,False,False,17.17,4.94,1.81,11.97,False


Split the `data` dataframe into 2 dataframes  `df_control`and `df_drugA` according to the value of `drugA`.

In [32]:
df_control = data[data['drugA'] == False]
df_drugA = data[data['drugA']]

Check the size of both dataframe using `dataframe.shape()`

In [33]:
## Control dataframe
df_control.shape

(8900, 10)

In [34]:
## DrugA dataframe
df_control.shape

(8900, 10)

<a id="stat"></a>
# 2. Descriptive statistics

Let's compute base statistics for our features in order to catch a glimpse of our population characteristics by separating dead vs alive patients. We will compute :
- For quantitative features : mean (std) and student t-test
- For qualitative features : % of prevalence and Chi-sq test

*Assuming that we are respecting the initial conditions of each test (iid gaussian distribution for quanti feats).*

We are going to use the `df.groupby().agg(agg_dict)` function. Check out the doc [here](https://pandas.pydata.org/pandas-docs/version/0.22/generated/pandas.core.groupby.DataFrameGroupBy.agg.html)  
We will therefore need to provide for each column of `data` the desired aggregation function.

Create a `prevalence` function that returns the percentage of True values from a list of booleans.

In [35]:
def prevalence(values):
    return round(values.sum()*100 / len(values),2)

Create a `mediane_IQR` function that returns the median and the Q1-Q3 of patients as a string

In [1]:
def mediane_IQR(values):
    return str(round(values.median(), 2)) + ' (' + str(round(values.quantile(0.25), 2)) + '-' +  str(round(values.quantile(0.75), 2)) + ')'

Create the "agg_dict" dictionary storing for each column name (keys) the list of transforming functions (values). You can use the `np.mean` and `np.std` registered functions.

TIP : Example
```python
agg_dict = {
    "person_id": ["count"],
    "age": [mediane_IQR],
    ...
}
```

In [37]:
agg_dict = {
    "person_id": ["count"],
    "age": [mediane_IQR],
    "male": [prevalence],
    "history_cancer" : [prevalence],
    "diabetes" : [prevalence],
    "bmi" : [mediane_IQR],
    "crp": [mediane_IQR],
    "hb": [mediane_IQR],
    "urea": [mediane_IQR],
}

Create a summary DataFrame from `data` using the predefined `agg_dict` to compare base statistics between the cohort taking drugA and the control cohort.

In [38]:
data_summary = data.groupby("drugA").agg(agg_dict).T

In [39]:
data_summary

,drugA,False,True
person_id,count,8900,10000
age,mediane_IQR,55.32 (33.12-77.76),49.88 (25.02-74.69)
male,prevalence,50.0,50.0
history_cancer,prevalence,1.73,1.42
diabetes,prevalence,10.42,9.39
bmi,mediane_IQR,21.64 (19.86-23.58),21.43 (19.5-23.39)
crp,mediane_IQR,4.5 (4.35-4.67),4.51 (4.35-4.68)
hb,mediane_IQR,13.45 (12.39-14.51),13.36 (12.29-14.45)
urea,mediane_IQR,2.01 (1.75-2.25),1.97 (1.71-2.23)


Compute statistical tests to assess the significancy of the resulting difference between the dead and the living groups.  
For numerical values (age and bio), you can compute the p-values associated to Student t-tests (function `ttest_ind`, doc [here](https://docs.scipy.org/doc/scipy/reference/generated/scipy.stats.ttest_ind.html)), and for categorical values (gender, comorbidities and drug) chi-sq tests (function `chi2_contigency`, doc [here](https://docs.scipy.org/doc/scipy/reference/generated/scipy.stats.chi2_contingency.html)).

TIP : 
You may need to use the two seperated dataframes `df_control` and `df_drugA` for the t-test

In [40]:
numerical = ["age", "bmi", "crp", "hb", "urea"]
test_categorical = ["male", "history_cancer", "diabetes", "drugB"]

In [41]:
for feat, sub_value in data_summary.index:
    if feat in numerical:
        data_summary.loc[(feat,sub_value),"t-test"] = round(ttest_ind(df_control[feat], df_drugA[feat])[1],4)

    if feat in test_categorical:
        data_summary.loc[(feat,sub_value),"chi-sq"] = round(chi2_contingency(pd.crosstab(data["drugA"], data[feat]))[1],4)

In [ ]:
data_summary

,drugA,False,True,t-test,chi-sq
person_id,count,8900,10000,NaN,NaN
age,mediane_IQR,55.32 (33.12-77.76),49.88 (25.02-74.69),0.0000,NaN
male,prevalence,50.0,50.0,NaN,1.0000
history_cancer,prevalence,1.73,1.42,NaN,0.0976
diabetes,prevalence,10.42,9.39,NaN,0.0195
bmi,mediane_IQR,21.64 (19.86-23.58),21.43 (19.5-23.39),0.0000,NaN
crp,mediane_IQR,4.5 (4.35-4.67),4.51 (4.35-4.68),0.0003,NaN
hb,mediane_IQR,13.45 (12.39-14.51),13.36 (12.29-14.45),0.0000,NaN
urea,mediane_IQR,2.01 (1.75-2.25),1.97 (1.71-2.23),0.0000,NaN


Regroup the statistical test results in one column <br>
Rename the False and True columns to Cohorte Control and Cohorte avec DrugA respectively

In [44]:
data_summary['p_value'] = data_summary['t-test'].fillna(data_summary['chi-sq'])
data_summary['p_value'] = data_summary['p_value'].fillna('')

In [45]:
data_summary = data_summary.rename(columns = {False : 'Cohorte Control', True : 'Cohorte avec DrugA'})

In [46]:
data_summary[['Cohorte Control', 'Cohorte avec DrugA', 'p_value']]

,drugA,Cohorte Control,Cohorte avec DrugA,p_value
person_id,count,8900,10000,
age,mediane_IQR,55.32 (33.12-77.76),49.88 (25.02-74.69),0.0
male,prevalence,50.0,50.0,1.0
history_cancer,prevalence,1.73,1.42,0.0976
diabetes,prevalence,10.42,9.39,0.0195
bmi,mediane_IQR,21.64 (19.86-23.58),21.43 (19.5-23.39),0.0
crp,mediane_IQR,4.5 (4.35-4.67),4.51 (4.35-4.68),0.0003
hb,mediane_IQR,13.45 (12.39-14.51),13.36 (12.29-14.45),0.0
urea,mediane_IQR,2.01 (1.75-2.25),1.97 (1.71-2.23),0.0


What can you conclude ?

**Correction**:
<br>The population are different. The patient taking drugA seem to be younger and have less comorbidities that the patients from the control cohort. 

# 3. Takeaways

Cohorts receiving specific treatments can be selected for their good health. It is very important to make sure the control cohort and the specific cohort are similar in order to be able to draw a significant conclusion of the effect of a treatment. 

It is of paramount importance to work closely with clinician to be aware of such biases and avoid over-interpretation, as long as performing stratified analyses to have a broader view of the effects.